In [4]:
import nltk
import os
import re
import math
import operator
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
#nltk.download('averaged_perceptron_tagger')
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()

In [5]:
def lemmatize_words(words):
    lemmatized_words = []
    for word in words:
       lemmatized_words.append(wordlemmatizer.lemmatize(word))
    return lemmatized_words

In [6]:
def stem_words(words):
    stemmed_words = []
    for word in words:
       stemmed_words.append(stemmer.stem(word))
    return stemmed_words

In [7]:
def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text

In [8]:

def freq(words):
    words = [word.lower() for word in words]
    dict_freq = {}
    words_unique = []
    for word in words:
       if word not in words_unique:
           words_unique.append(word)
    for word in words_unique:
       dict_freq[word] = words.count(word)
    return dict_freq

In [9]:
def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    pos_tagged_noun_verb = []
    for word,tag in pos_tag:
        if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
             pos_tagged_noun_verb.append(word)
    return pos_tagged_noun_verb

In [10]:
def tf_score(word,sentence):
    #print("****************************Calculate Tf_Score*************************************")
    #print("Calcualte tf_score for the \nword ={0}  \nsentence = {1}".format(word,sentence))
    freq_sum = 0
    word_frequency_in_sentence = 0
    len_sentence = len(sentence)
    #print("length of the sentence =",len_sentence)
    for word_in_sentence in sentence.split():
        if word == word_in_sentence:
            word_frequency_in_sentence = word_frequency_in_sentence + 1
    #print("word_frequency_in_sentence =",word_frequency_in_sentence)
    tf =  word_frequency_in_sentence/ len_sentence
    #print("tf = {0} / {1}".fomrat(word_frequency_in_sentence,len_sentence))
    return tf

In [11]:
def idf_score(no_of_sentences,word,sentences):
    #print("***************************************Calculating Idf_Score****************************")
    #print("no_of_sentences = ",no_of_sentences)
    #print("word = ",word)
    no_of_sentence_containing_word = 0
    for sentence in sentences:
        sentence = remove_special_characters(str(sentence))
        sentence = re.sub(r'\d+', '', sentence)
        sentence = sentence.split()
        sentence = [word for word in sentence if word.lower() not in Stopwords and len(word)>1]
        sentence = [word.lower() for word in sentence]
        sentence = [wordlemmatizer.lemmatize(word) for word in sentence]
        if word in sentence:
            no_of_sentence_containing_word = no_of_sentence_containing_word + 1
        #print("no_of_sentence_containing_word=",no_of_sentence_containing_word)
    idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
    return idf

In [12]:
def tf_idf_score(tf,idf):
    #print("***************************Calculate Tf_idf_score***********************")
    #print("tf_idf_score : {0} * {1} ={2}".format(tf,idf,tf*idf))
    return tf*idf

In [13]:
def word_tfidf(dict_freq,word,sentences,sentence):
    word_tfidf = []
    #print("dict_freq = ",dict_freq)
    print("word:",word)
    tf = tf_score(word,sentence)
    print("tf_score:",tf)
    idf = idf_score(len(sentences),word,sentences)
    print("idf=",idf)
    tf_idf = tf_idf_score(tf,idf)
    print("tf_idf =",tf_idf)
    return tf_idf

In [14]:
def sentence_importance(sentence,dict_freq,sentences):
    sentence_score = 0
    sentence = remove_special_characters(str(sentence)) 
    
    sentence = re.sub(r'\d+', '', sentence)
    pos_tagged_sentence = [] 
    print("senetcne************************************************")
    print(sentence)
    no_of_sentences = len(sentences)
    print("no_of_sentences = ",no_of_sentences)
    pos_tagged_sentence = pos_tagging(sentence)
    print("pos_tagged_sentence = ",pos_tagged_sentence)
    for word in pos_tagged_sentence:
        print("len(word) =",len(word))
        if word.lower() not in Stopwords and word not in Stopwords and len(word)>1: 
            word = word.lower()
            word = wordlemmatizer.lemmatize(word)
            print("wordlemmatizer:",word)
            sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
            
            print('sentence_score :{0} = {1}'.format(word,sentence_score))
    return sentence_score

In [15]:
file = 'SampleText01.txt'
file = open(file , 'r')
text = file.read()
print(text)
tokenized_sentence = sent_tokenize(text)

print("tokenized_sentence : \n",tokenized_sentence)
text = remove_special_characters(str(text))

text = re.sub(r'\d+', '', text)

tokenized_words_with_stopwords = word_tokenize(text)

tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]

tokenized_words = [word for word in tokenized_words if len(word) > 1]

tokenized_words = [word.lower() for word in tokenized_words]

tokenized_words = lemmatize_words(tokenized_words)

print("tokenized_words = \n",tokenized_words)

word_freq = freq(tokenized_words)

print("word_freq: \n",word_freq)

input_user = int(input('Percentage of information to retain(in percent):'))

no_of_sentences = int((input_user * len(tokenized_sentence))/100)

print(no_of_sentences)

c = 1

sentence_with_importance = {}

for sent in tokenized_sentence:
    sentenceimp = sentence_importance(sent,word_freq,tokenized_sentence)
    
    sentence_with_importance[c] = sentenceimp
    c = c+1
    
sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)

cnt = 0
summary = []
sentence_no = []

for word_prob in sentence_with_importance:
    if cnt < no_of_sentences:
        sentence_no.append(word_prob[0])
        cnt = cnt+1
    else:
      break
sentence_no.sort()
cnt = 1

for sentence in tokenized_sentence:
    if cnt in sentence_no:
       summary.append(sentence)
    cnt = cnt+1
summary = " ".join(summary)

print("\n")
print("Summary:")
print(summary)
outF = open('summary.txt',"w")
outF.write(summary)

In an attempt to build an AI-ready workforce, Microsoft announced Intelligent Cloud Hub which has been launched to empower the next generation of students with AI-ready skills. Envisioned as a three-year collaborative program, Intelligent Cloud Hub will support around 100 institutions with AI infrastructure, course content and curriculum, developer support, development tools and give students access to cloud and AI services. As part of the program, the Redmond giant which wants to expand its reach and is planning to build a strong developer ecosystem in India with the program will set up the core AI infrastructure and IoT Hub for the selected campuses. The company will provide AI development tools and Azure AI services such as Microsoft Cognitive Services, Bot Services and Azure Machine Learning.According to Manish Prakash, Country General Manager-PS, Health and Education, Microsoft India, said, "With AI being the defining technology of our time, it is transforming lives and industry a

pos_tagged_sentence =  ['Envisioned', 'program', 'Intelligent', 'Cloud', 'Hub', 'support', 'institutions', 'AI', 'infrastructure', 'course', 'content', 'curriculum', 'developer', 'support', 'development', 'tools', 'students', 'access', 'cloud', 'AI', 'services']
len(word) = 10
wordlemmatizer: envisioned
word: envisioned
tf_score: 0.0
idf= 1.0413926851582251
tf_idf = 0.0
sentence_score :envisioned = 0.0
len(word) = 7
wordlemmatizer: program
word: program
tf_score: 0.004132231404958678
idf= 0.1962946451439682
tf_idf = 0.0008111348972891248
sentence_score :program = 0.0008111348972891248
len(word) = 11
wordlemmatizer: intelligent
word: intelligent
tf_score: 0.0
idf= 0.5642714304385625
tf_idf = 0.0
sentence_score :intelligent = 0.0008111348972891248
len(word) = 5
wordlemmatizer: cloud
word: cloud
tf_score: 0.004132231404958678
idf= 0.43933269383026263
tf_idf = 0.0018154243546705069
sentence_score :cloud = 0.0026265592519596316
len(word) = 3
wordlemmatizer: hub
word: hub
tf_score: 0.0
idf= 

idf= 1.0413926851582251
tf_idf = 0.014877038359403216
sentence_score :working = 0.040330686568724204
len(word) = 2
wordlemmatizer: ai
word: ai
tf_score: 0.0
idf= 0.13830269816628146
tf_idf = 0.0
sentence_score :ai = 0.040330686568724204
senetcne************************************************
Thats why it has become more critical than ever for educational institutions to integrate new cloud and AI technologies
no_of_sentences =  11
pos_tagged_sentence =  ['Thats', 'has', 'become', 'institutions', 'integrate', 'cloud', 'AI', 'technologies']
len(word) = 5
wordlemmatizer: thats
word: thats
tf_score: 0.0
idf= 1.0413926851582251
tf_idf = 0.0
sentence_score :thats = 0.0
len(word) = 3
len(word) = 6
wordlemmatizer: become
word: become
tf_score: 0.008403361344537815
idf= 1.0413926851582251
tf_idf = 0.008751199034943069
sentence_score :become = 0.008751199034943069
len(word) = 12
wordlemmatizer: institution
word: institution
tf_score: 0.0
idf= 0.7403626894942439
tf_idf = 0.0
sentence_score :inst

idf= 1.0413926851582251
tf_idf = 0.004866320958683295
sentence_score :developed = 0.005783585655617725
len(word) = 7
wordlemmatizer: provide
word: provide
tf_score: 0.004672897196261682
idf= 0.7403626894942439
tf_idf = 0.0034596387359544104
sentence_score :provide = 0.009243224391572135
len(word) = 3
wordlemmatizer: job
word: job
tf_score: 0.004672897196261682
idf= 0.7403626894942439
tf_idf = 0.0034596387359544104
sentence_score :job = 0.012702863127526547
len(word) = 6
wordlemmatizer: skill
word: skill
tf_score: 0.0
idf= 0.43933269383026263
tf_idf = 0.0
sentence_score :skill = 0.012702863127526547
len(word) = 11
wordlemmatizer: programmer
word: programmer
tf_score: 0.0
idf= 1.0413926851582251
tf_idf = 0.0
sentence_score :programmer = 0.012702863127526547
len(word) = 6
wordlemmatizer: wanted
word: wanted
tf_score: 0.004672897196261682
idf= 1.0413926851582251
tf_idf = 0.004866320958683295
sentence_score :wanted = 0.01756918408620984
len(word) = 4
wordlemmatizer: hone
word: hone
tf_score

812